# 🤖 Xây Dựng và Đánh Giá Mô Hình Machine Learning

## Mục tiêu:
- Xây dựng và so sánh nhiều mô hình ML
- Tối ưu hóa hyperparameters
- Đánh giá hiệu suất với các metrics phù hợp
- Phân tích feature importance
- Chọn mô hình tốt nhất

In [1]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Model Selection và Evaluation
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report, roc_curve, precision_recall_curve
)

# Cấu hình hiển thị
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
np.random.seed(42)

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <B637898E-C0C3-3F93-8C08-800EE41A7A5B> /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/loc

## 1. Tải Dữ Liệu Đã Xử Lý

In [ ]:
# Tải các version dữ liệu khác nhau
def load_data(filename):
    with open(f'../data/processed/{filename}', 'rb') as f:
        return pickle.load(f)

# Tải tất cả versions
data_original = load_data('data_original.pkl')
data_smote = load_data('data_smote.pkl')
data_under = load_data('data_under.pkl')
data_smoteenn = load_data('data_smoteenn.pkl')

print("📊 DỮ LIỆU ĐÃ TẢI:")
datasets = {
    'Original': data_original,
    'SMOTE': data_smote,
    'Under Sampling': data_under,
    'SMOTEENN': data_smoteenn
}

for name, data in datasets.items():
    X_train, y_train = data['X_train'], data['y_train']
    stroke_rate = np.mean(y_train)
    print(f"{name:15} - Train: {X_train.shape[0]:5} samples - Stroke rate: {stroke_rate:.3f}")

print(f"\n✅ Features: {len(data_original['feature_names'])}")
print(f"✅ Test samples: {data_original['X_test'].shape[0]}")

## 2. Định Nghĩa Các Mô Hình

In [ ]:
# Định nghĩa các mô hình với hyperparameters cơ bản
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(random_state=42, verbose=-1),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42)
}

print(f"🤖 ĐÃ ĐỊNH NGHĨA {len(models)} MÔ HÌNH:")
for i, name in enumerate(models.keys(), 1):
    print(f"{i:2}. {name}")

## 3. Hàm Đánh Giá Mô Hình

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    """
    Đánh giá toàn diện một mô hình
    """
    # Training
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None
    }
    
    return model, metrics, y_pred, y_pred_proba

def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
    """
    Vẽ confusion matrix
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['No Stroke', 'Stroke'],
                yticklabels=['No Stroke', 'Stroke'])
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

def plot_roc_curve(y_true, y_pred_proba, title="ROC Curve"):
    """
    Vẽ ROC curve
    """
    if y_pred_proba is None:
        print("Không có probability predictions để vẽ ROC curve")
        return
    
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    auc = roc_auc_score(y_true, y_pred_proba)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

print("✅ Đã định nghĩa các hàm đánh giá mô hình")

## 4. So Sánh Mô Hình Trên Dữ Liệu Gốc

In [ ]:
# Sử dụng dữ liệu gốc để so sánh ban đầu
X_train = data_original['X_train']
X_test = data_original['X_test']
y_train = data_original['y_train']
y_test = data_original['y_test']

print("🔄 ĐANG TRAINING VÀ ĐÁNH GIÁ CÁC MÔ HÌNH...")
print("(Có thể mất vài phút)\n")

# Lưu kết quả
results = {}
trained_models = {}

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"🔄 Training {name}...")
    
    try:
        # Cross-validation scores
        cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
        
        # Train và evaluate trên test set
        trained_model, metrics, y_pred, y_pred_proba = evaluate_model(
            model, X_train, X_test, y_train, y_test, name
        )
        
        # Lưu kết quả
        results[name] = {
            'cv_auc_mean': cv_scores.mean(),
            'cv_auc_std': cv_scores.std(),
            'test_metrics': metrics,
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        trained_models[name] = trained_model
        
        print(f"   ✅ CV AUC: {cv_scores.mean():.3f} (±{cv_scores.std():.3f})")
        print(f"   ✅ Test AUC: {metrics['roc_auc']:.3f if metrics['roc_auc'] else 'N/A'}")
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        continue

print(f"\n✅ Hoàn thành training {len(results)} mô hình")

## 5. Bảng So Sánh Kết Quả

In [ ]:
# Tạo bảng so sánh
comparison_data = []

for name, result in results.items():
    metrics = result['test_metrics']
    comparison_data.append({
        'Model': name,
        'CV AUC': f"{result['cv_auc_mean']:.3f} ± {result['cv_auc_std']:.3f}",
        'Test AUC': f"{metrics['roc_auc']:.3f}" if metrics['roc_auc'] else 'N/A',
        'Accuracy': f"{metrics['accuracy']:.3f}",
        'Precision': f"{metrics['precision']:.3f}",
        'Recall': f"{metrics['recall']:.3f}",
        'F1-Score': f"{metrics['f1']:.3f}"
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Test AUC', ascending=False)

print("📊 BẢNG SO SÁNH KẾT QUẢ CÁC MÔ HÌNH:")
print("=" * 100)
print(comparison_df.to_string(index=False))
print("=" * 100)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# AUC scores
auc_scores = [results[name]['test_metrics']['roc_auc'] for name in results.keys() 
              if results[name]['test_metrics']['roc_auc'] is not None]
model_names = [name for name in results.keys() 
               if results[name]['test_metrics']['roc_auc'] is not None]

axes[0,0].barh(model_names, auc_scores, color='skyblue')
axes[0,0].set_xlabel('AUC Score')
axes[0,0].set_title('Test AUC Scores')
axes[0,0].set_xlim(0, 1)

# F1 scores
f1_scores = [results[name]['test_metrics']['f1'] for name in results.keys()]
axes[0,1].barh(list(results.keys()), f1_scores, color='lightgreen')
axes[0,1].set_xlabel('F1 Score')
axes[0,1].set_title('F1 Scores')
axes[0,1].set_xlim(0, 1)

# Precision vs Recall
precisions = [results[name]['test_metrics']['precision'] for name in results.keys()]
recalls = [results[name]['test_metrics']['recall'] for name in results.keys()]

axes[1,0].scatter(recalls, precisions, s=100, alpha=0.7)
for i, name in enumerate(results.keys()):
    axes[1,0].annotate(name, (recalls[i], precisions[i]), 
                      xytext=(5, 5), textcoords='offset points', fontsize=8)
axes[1,0].set_xlabel('Recall')
axes[1,0].set_ylabel('Precision')
axes[1,0].set_title('Precision vs Recall')
axes[1,0].grid(True, alpha=0.3)

# CV AUC với error bars
cv_means = [results[name]['cv_auc_mean'] for name in results.keys()]
cv_stds = [results[name]['cv_auc_std'] for name in results.keys()]

axes[1,1].barh(list(results.keys()), cv_means, xerr=cv_stds, 
               color='salmon', alpha=0.7, capsize=5)
axes[1,1].set_xlabel('CV AUC Score')
axes[1,1].set_title('Cross-Validation AUC Scores')
axes[1,1].set_xlim(0, 1)

plt.tight_layout()
plt.show()

## 6. Phân Tích Top 3 Mô Hình

In [ ]:
# Chọn top 3 mô hình dựa trên AUC
auc_results = [(name, results[name]['test_metrics']['roc_auc']) 
               for name in results.keys() 
               if results[name]['test_metrics']['roc_auc'] is not None]
auc_results.sort(key=lambda x: x[1], reverse=True)
top_3_models = [name for name, _ in auc_results[:3]]

print(f"🏆 TOP 3 MÔ HÌNH THEO AUC:")
for i, name in enumerate(top_3_models, 1):
    auc = results[name]['test_metrics']['roc_auc']
    print(f"{i}. {name}: {auc:.3f}")

# Phân tích chi tiết top 3
fig, axes = plt.subplots(len(top_3_models), 3, figsize=(18, 6*len(top_3_models)))
if len(top_3_models) == 1:
    axes = axes.reshape(1, -1)

for i, model_name in enumerate(top_3_models):
    y_pred = results[model_name]['predictions']
    y_pred_proba = results[model_name]['probabilities']
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i,0],
                xticklabels=['No Stroke', 'Stroke'],
                yticklabels=['No Stroke', 'Stroke'])
    axes[i,0].set_title(f'{model_name} - Confusion Matrix')
    axes[i,0].set_xlabel('Predicted')
    axes[i,0].set_ylabel('Actual')
    
    # ROC Curve
    if y_pred_proba is not None:
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        auc = roc_auc_score(y_test, y_pred_proba)
        axes[i,1].plot(fpr, tpr, color='darkorange', lw=2, 
                      label=f'ROC curve (AUC = {auc:.3f})')
        axes[i,1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        axes[i,1].set_xlim([0.0, 1.0])
        axes[i,1].set_ylim([0.0, 1.05])
        axes[i,1].set_xlabel('False Positive Rate')
        axes[i,1].set_ylabel('True Positive Rate')
        axes[i,1].set_title(f'{model_name} - ROC Curve')
        axes[i,1].legend(loc="lower right")
        axes[i,1].grid(True, alpha=0.3)
        
        # Precision-Recall Curve
        precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
        axes[i,2].plot(recall, precision, color='blue', lw=2)
        axes[i,2].set_xlabel('Recall')
        axes[i,2].set_ylabel('Precision')
        axes[i,2].set_title(f'{model_name} - Precision-Recall Curve')
        axes[i,2].grid(True, alpha=0.3)
    else:
        axes[i,1].text(0.5, 0.5, 'No probability\npredictions available', 
                      ha='center', va='center', transform=axes[i,1].transAxes)
        axes[i,2].text(0.5, 0.5, 'No probability\npredictions available', 
                      ha='center', va='center', transform=axes[i,2].transAxes)

plt.tight_layout()
plt.show()

# Classification reports
print("\n📋 CLASSIFICATION REPORTS:")
print("=" * 80)
for model_name in top_3_models:
    y_pred = results[model_name]['predictions']
    print(f"\n{model_name.upper()}:")
    print("-" * 40)
    print(classification_report(y_test, y_pred, target_names=['No Stroke', 'Stroke']))

## 7. Feature Importance Analysis

In [ ]:
# Phân tích feature importance cho các mô hình tree-based
tree_based_models = ['Random Forest', 'Gradient Boosting', 'XGBoost', 'LightGBM']
feature_names = data_original['feature_names']

fig, axes = plt.subplots(2, 2, figsize=(20, 15))
axes = axes.ravel()

for i, model_name in enumerate(tree_based_models):
    if model_name in trained_models:
        model = trained_models[model_name]
        
        # Lấy feature importance
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            
            # Tạo DataFrame và sort
            feature_imp = pd.DataFrame({
                'feature': feature_names,
                'importance': importances
            }).sort_values('importance', ascending=True)
            
            # Lấy top 15 features
            top_features = feature_imp.tail(15)
            
            # Plot
            axes[i].barh(top_features['feature'], top_features['importance'], 
                        color='skyblue', alpha=0.8)
            axes[i].set_title(f'{model_name} - Top 15 Feature Importance')
            axes[i].set_xlabel('Importance')
            axes[i].tick_params(axis='y', labelsize=8)
            
            # In top 10 features
            print(f"\n🔍 {model_name.upper()} - TOP 10 FEATURES:")
            print("-" * 50)
            for idx, row in feature_imp.tail(10).iterrows():
                print(f"{row['feature']:30} {row['importance']:.4f}")

plt.tight_layout()
plt.show()

## 8. Hyperparameter Tuning cho Mô Hình Tốt Nhất

In [ ]:
# Chọn mô hình tốt nhất để tune
best_model_name = top_3_models[0]
print(f"🎯 HYPERPARAMETER TUNING CHO: {best_model_name}")

# Định nghĩa parameter grids
param_grids = {
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    },
    'LightGBM': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'num_leaves': [31, 50, 100],
        'subsample': [0.8, 0.9, 1.0]
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
    },
    'Logistic Regression': {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    }
}

if best_model_name in param_grids:
    print(f"🔄 Đang thực hiện RandomizedSearchCV...")
    
    # Tạo mô hình base
    base_model = models[best_model_name]
    
    # RandomizedSearchCV
    random_search = RandomizedSearchCV(
        base_model,
        param_grids[best_model_name],
        n_iter=50,  # Số lần thử
        cv=5,
        scoring='roc_auc',
        random_state=42,
        n_jobs=-1,
        verbose=1
    )
    
    # Fit
    random_search.fit(X_train, y_train)
    
    # Kết quả
    print(f"\n✅ BEST PARAMETERS:")
    for param, value in random_search.best_params_.items():
        print(f"{param:20}: {value}")
    
    print(f"\n📊 BEST CV SCORE: {random_search.best_score_:.4f}")
    
    # Đánh giá mô hình đã tune
    best_tuned_model = random_search.best_estimator_
    tuned_model, tuned_metrics, tuned_pred, tuned_proba = evaluate_model(
        best_tuned_model, X_train, X_test, y_train, y_test, f"{best_model_name} (Tuned)"
    )
    
    # So sánh với mô hình gốc
    original_metrics = results[best_model_name]['test_metrics']
    
    print(f"\n📈 SO SÁNH KẾT QUẢ:")
    print("-" * 60)
    print(f"{'Metric':<15} {'Original':<12} {'Tuned':<12} {'Improvement':<12}")
    print("-" * 60)
    
    for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
        if original_metrics[metric] is not None and tuned_metrics[metric] is not None:
            orig_val = original_metrics[metric]
            tuned_val = tuned_metrics[metric]
            improvement = tuned_val - orig_val
            print(f"{metric:<15} {orig_val:<12.4f} {tuned_val:<12.4f} {improvement:+.4f}")
    
    # Lưu mô hình tốt nhất
    best_final_model = best_tuned_model
    
else:
    print(f"⚠️ Không có parameter grid cho {best_model_name}")
    best_final_model = trained_models[best_model_name]

## 9. Thử Nghiệm với Dữ Liệu SMOTE

In [ ]:
print("🔄 THỰC NGHIỆM VỚI DỮ LIỆU SMOTE:")

# Sử dụng dữ liệu SMOTE
X_train_smote = data_smote['X_train']
y_train_smote = data_smote['y_train']
X_test_smote = data_smote['X_test']  # Test set giữ nguyên
y_test_smote = data_smote['y_test']

print(f"Train set size: {X_train_smote.shape[0]} (SMOTE)")
print(f"Stroke ratio: {np.mean(y_train_smote):.3f}")

# Test top 3 models với SMOTE data
smote_results = {}

for model_name in top_3_models:
    print(f"\n🔄 Testing {model_name} with SMOTE data...")
    
    # Tạo mô hình mới
    model = models[model_name]
    
    # Train và evaluate
    trained_model, metrics, y_pred, y_pred_proba = evaluate_model(
        model, X_train_smote, X_test_smote, y_train_smote, y_test_smote, 
        f"{model_name} (SMOTE)"
    )
    
    smote_results[model_name] = {
        'metrics': metrics,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print(f"   AUC: {metrics['roc_auc']:.4f}")
    print(f"   F1:  {metrics['f1']:.4f}")

# So sánh kết quả Original vs SMOTE
print("\n📊 SO SÁNH ORIGINAL VS SMOTE:")
print("=" * 80)
print(f"{'Model':<20} {'Original AUC':<15} {'SMOTE AUC':<15} {'Original F1':<15} {'SMOTE F1':<15}")
print("=" * 80)

for model_name in top_3_models:
    orig_auc = results[model_name]['test_metrics']['roc_auc']
    smote_auc = smote_results[model_name]['metrics']['roc_auc']
    orig_f1 = results[model_name]['test_metrics']['f1']
    smote_f1 = smote_results[model_name]['metrics']['f1']
    
    print(f"{model_name:<20} {orig_auc:<15.4f} {smote_auc:<15.4f} {orig_f1:<15.4f} {smote_f1:<15.4f}")

# Visualization comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# AUC comparison
model_names = list(top_3_models)
original_aucs = [results[name]['test_metrics']['roc_auc'] for name in model_names]
smote_aucs = [smote_results[name]['metrics']['roc_auc'] for name in model_names]

x = np.arange(len(model_names))
width = 0.35

axes[0].bar(x - width/2, original_aucs, width, label='Original', alpha=0.8, color='skyblue')
axes[0].bar(x + width/2, smote_aucs, width, label='SMOTE', alpha=0.8, color='lightgreen')
axes[0].set_xlabel('Models')
axes[0].set_ylabel('AUC Score')
axes[0].set_title('AUC Comparison: Original vs SMOTE')
axes[0].set_xticks(x)
axes[0].set_xticklabels(model_names, rotation=45)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# F1 comparison
original_f1s = [results[name]['test_metrics']['f1'] for name in model_names]
smote_f1s = [smote_results[name]['metrics']['f1'] for name in model_names]

axes[1].bar(x - width/2, original_f1s, width, label='Original', alpha=0.8, color='salmon')
axes[1].bar(x + width/2, smote_f1s, width, label='SMOTE', alpha=0.8, color='gold')
axes[1].set_xlabel('Models')
axes[1].set_ylabel('F1 Score')
axes[1].set_title('F1 Comparison: Original vs SMOTE')
axes[1].set_xticks(x)
axes[1].set_xticklabels(model_names, rotation=45)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Lưu Mô Hình Tốt Nhất

In [ ]:
# Xác định mô hình tốt nhất cuối cùng
print("💾 LƯU MÔ HÌNH TỐT NHẤT:")

# So sánh tất cả các version
final_comparison = []

# Original models
for name in top_3_models:
    metrics = results[name]['test_metrics']
    final_comparison.append({
        'model_name': f"{name} (Original)",
        'auc': metrics['roc_auc'],
        'f1': metrics['f1'],
        'precision': metrics['precision'],
        'recall': metrics['recall']
    })

# SMOTE models
for name in top_3_models:
    metrics = smote_results[name]['metrics']
    final_comparison.append({
        'model_name': f"{name} (SMOTE)",
        'auc': metrics['roc_auc'],
        'f1': metrics['f1'],
        'precision': metrics['precision'],
        'recall': metrics['recall']
    })

# Tuned model (nếu có)
if 'tuned_metrics' in locals():
    final_comparison.append({
        'model_name': f"{best_model_name} (Tuned)",
        'auc': tuned_metrics['roc_auc'],
        'f1': tuned_metrics['f1'],
        'precision': tuned_metrics['precision'],
        'recall': tuned_metrics['recall']
    })

# Tìm mô hình tốt nhất dựa trên AUC
best_overall = max(final_comparison, key=lambda x: x['auc'] if x['auc'] else 0)

print(f"🏆 MÔ HÌNH TỐT NHẤT: {best_overall['model_name']}")
print(f"   AUC: {best_overall['auc']:.4f}")
print(f"   F1:  {best_overall['f1']:.4f}")
print(f"   Precision: {best_overall['precision']:.4f}")
print(f"   Recall: {best_overall['recall']:.4f}")

# Lưu mô hình và metadata
import os
os.makedirs('../models', exist_ok=True)

# Xác định mô hình để lưu
if 'Tuned' in best_overall['model_name']:
    model_to_save = best_final_model
    data_version = 'original'
elif 'SMOTE' in best_overall['model_name']:
    # Train lại mô hình tốt nhất với SMOTE data
    model_name_clean = best_overall['model_name'].replace(' (SMOTE)', '')
    model_to_save = models[model_name_clean]
    model_to_save.fit(X_train_smote, y_train_smote)
    data_version = 'smote'
else:
    model_name_clean = best_overall['model_name'].replace(' (Original)', '')
    model_to_save = trained_models[model_name_clean]
    data_version = 'original'

# Lưu mô hình
model_info = {
    'model': model_to_save,
    'model_name': best_overall['model_name'],
    'data_version': data_version,
    'metrics': {
        'auc': best_overall['auc'],
        'f1': best_overall['f1'],
        'precision': best_overall['precision'],
        'recall': best_overall['recall']
    },
    'feature_names': data_original['feature_names'],
    'scaler': data_original['scaler']
}

with open('../models/best_stroke_model.pkl', 'wb') as f:
    pickle.dump(model_info, f)

print(f"\n✅ Đã lưu mô hình tại: ../models/best_stroke_model.pkl")

# Lưu tất cả kết quả
all_results = {
    'original_results': results,
    'smote_results': smote_results,
    'best_model_info': model_info,
    'comparison_data': final_comparison
}

with open('../results/modeling_results.pkl', 'wb') as f:
    pickle.dump(all_results, f)

print(f"✅ Đã lưu tất cả kết quả tại: ../results/modeling_results.pkl")

## 11. Tóm Tắt Kết Quả Modeling

In [ ]:
print("""📋 TÓM TẮT QUÁ TRÌNH MODELING:

🤖 CÁC MÔ HÌNH ĐÃ THỰC NGHIỆM:
1. ✅ Logistic Regression
2. ✅ Random Forest  
3. ✅ Gradient Boosting
4. ✅ XGBoost
5. ✅ LightGBM
6. ✅ SVM
7. ✅ K-Nearest Neighbors
8. ✅ Naive Bayes
9. ✅ Decision Tree
10. ✅ AdaBoost

📊 PHƯƠNG PHÁP ĐÁNH GIÁ:
- ✅ 5-fold Cross Validation
- ✅ Multiple metrics: AUC, F1, Precision, Recall, Accuracy
- ✅ ROC Curves và Precision-Recall Curves
- ✅ Confusion Matrix analysis
- ✅ Feature Importance analysis

🔧 TECHNIQUES ĐÃ ÁP DỤNG:
- ✅ Hyperparameter tuning với RandomizedSearchCV
- ✅ So sánh Original vs SMOTE data
- ✅ Feature importance analysis cho tree-based models
- ✅ Comprehensive model comparison

🏆 KẾT QUẢ CUỐI CÙNG:
- Mô hình tốt nhất: {}
- AUC Score: {:.4f}
- F1 Score: {:.4f}
- Precision: {:.4f}
- Recall: {:.4f}

💾 FILES ĐÃ LƯU:
- best_stroke_model.pkl: Mô hình tốt nhất + metadata
- modeling_results.pkl: Tất cả kết quả thực nghiệm

🚀 SẴN SÀNG CHO DEPLOYMENT!
""".format(
    best_overall['model_name'],
    best_overall['auc'],
    best_overall['f1'],
    best_overall['precision'],
    best_overall['recall']
))